In [3]:
import math
from queue import PriorityQueue
import time

In [4]:
puzzle = 8
dim = int(math.sqrt(puzzle)) + 1
print(dim)

3


In [5]:
goal_state = []
co_ordinates = {}
for i in range(dim):
  temp = []
  for j in range(dim):
    val = i*dim+(j+1)
    temp.append(val)
    co_ord = [i, j]
    co_ordinates[val] = co_ord
  goal_state.append(temp)

last_val = goal_state[-1][-1]
goal_state[-1][-1] = 0
temp_co_ord = co_ordinates[last_val]
del co_ordinates[last_val]
co_ordinates[0] = temp_co_ord
print(last_val)
print(goal_state)
print(co_ordinates)

9
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
{1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [1, 0], 5: [1, 1], 6: [1, 2], 7: [2, 0], 8: [2, 1], 0: [2, 2]}


In [6]:
def manhattan_distance(state):
  m_dist = 0
  for i in range(dim):
    for j in range(dim):
      val = state[i][j]
      if(val == 0):
        continue
      goal_co_ord = co_ordinates[val]
      i_goal = goal_co_ord[0]
      j_goal = goal_co_ord[1]
      m_dist += abs(i_goal - i) + abs(j_goal - j)
  return m_dist

In [7]:
state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
dist = manhattan_distance(state)
print(dist)

14


In [8]:
def misplaced_tile(state):
  m_tile = 0
  for i in range(dim):
    for j in range(dim):
      val = goal_state[i][j]
      if(val == 0):
        continue
      if(val != state[i][j]):
        m_tile += 1
  return m_tile

In [9]:
state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
dist = misplaced_tile(state)
print(dist)

6


In [10]:
class Node:
  def __init__(self, state, parent=None, cost=0):
    self.state = state
    self.parent = parent
    self.cost = cost

  def __lt__(self, other_node):
        return self.cost < other_node.cost

In [11]:
def expand(node):
  state = node.state
  expanded_nodes = []
  blank_i = dim - 1
  blank_j = dim - 1

  for i in range(dim):
    for j in range(dim):
      if(state[i][j] == 0):
        blank_i = i
        blank_j = j
        break
  if(blank_i > 0):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i - 1][blank_j]
    child[blank_i][blank_j] = val
    child[blank_i - 1][blank_j] = 0
    child_node = Node(child, state, node.cost+1)
    expanded_nodes.append(child_node)

  if(blank_i < dim - 1):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i + 1][blank_j]
    child[blank_i][blank_j] = val
    child[blank_i + 1][blank_j] = 0
    child_node = Node(child, state, node.cost+1)
    expanded_nodes.append(child_node)

  if(blank_j > 0):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i][blank_j - 1]
    child[blank_i][blank_j] = val
    child[blank_i][blank_j - 1] = 0
    child_node = Node(child, state, node.cost+1)
    expanded_nodes.append(child_node)

  if(blank_j < dim - 1):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i][blank_j + 1]
    child[blank_i][blank_j] = val
    child[blank_i][blank_j + 1] = 0
    child_node = Node(child, state, node.cost+1)
    expanded_nodes.append(child_node)

  return expanded_nodes

In [12]:
state = [[1, 6, 7], [5, 0, 3], [4, 8, 2]]
node = Node(state)
exp_nodes = expand(node)
for node in exp_nodes:
  print(node.state)

[[1, 0, 7], [5, 6, 3], [4, 8, 2]]
[[1, 6, 7], [5, 8, 3], [4, 0, 2]]
[[1, 6, 7], [0, 5, 3], [4, 8, 2]]
[[1, 6, 7], [5, 3, 0], [4, 8, 2]]


In [13]:
def a_star_with_UC(node):
  q = PriorityQueue()
  q.put(node)
  is_visited = set()
  number_of_expansion = 0
  s_time = time.process_time()
  max_queue_size = 0
  while (q.empty() == False):
    nde = q.get()
    print(nde.state, 'cost: '+str(nde.cost))
    if(misplaced_tile(nde.state) == 0):
      e_time = time.process_time()
      print("reached the goal state!")
      print("the number of expansion required: "+ str(number_of_expansion))
      print("maximum queue size: "+ str(max_queue_size))
      print("time required: "+str(e_time - s_time))
      return nde
    expanded_nodes = expand(nde)
    number_of_expansion += 1
    tple = tuple(map(tuple, nde.state))
    is_visited.add(tple)

    for child_node in expanded_nodes:
      child_tuple = tuple(map(tuple, child_node.state))
      if(child_tuple not in is_visited):
        q.put(child_node)
    if(q.qsize() > max_queue_size):
      max_queue_size = q.qsize()
  print("queue is empty!")
  return None

In [ ]:
state = [[0, 7, 2], [4, 6, 1], [3, 5, 8]]
node = Node(state)
a_star_with_UC(node)

In [14]:
def a_star_with_MD(node):
  q = PriorityQueue()
  q.put(node)
  is_visited = set()
  number_of_expansion = 0
  s_time = time.process_time()
  max_queue_size = 0
  while (q.empty() == False):
    nde = q.get()
    print(nde.state, 'cost: '+str(nde.cost))
    if(misplaced_tile(nde.state) == 0):
      e_time = time.process_time()
      print("reached the goal state!")
      print("the number of expansion required: "+ str(number_of_expansion))
      print("maximum queue size: "+ str(max_queue_size))
      print("time required: "+str(e_time - s_time))
      return nde
    expanded_nodes = expand(nde)
    number_of_expansion += 1
    tple = tuple(map(tuple, nde.state))
    is_visited.add(tple)

    for child_node in expanded_nodes:
      child_tuple = tuple(map(tuple, child_node.state))
      if(child_tuple not in is_visited):
        child_node.cost = child_node.cost + manhattan_distance(child_node.state)
        q.put(child_node)
    if(q.qsize() > max_queue_size):
      max_queue_size = q.qsize()
  print("queue is empty!")
  return None

In [ ]:
state = [[0, 7, 2], [4, 6, 1], [3, 5, 8]]
node = Node(state)
a_star_with_MD(node)

In [15]:
def a_star_with_MT(node):
  q = PriorityQueue()
  q.put(node)
  is_visited = set()
  number_of_expansion = 0
  s_time = time.process_time()
  max_queue_size = 0
  while (q.empty() == False):
    nde = q.get()
    print(nde.state, 'cost: '+str(nde.cost))
    if(misplaced_tile(nde.state) == 0):
      e_time = time.process_time()
      print("reached the goal state!")
      print("the number of expansion required: "+ str(number_of_expansion))
      print("maximum queue size: "+ str(max_queue_size))
      print("time required: "+str(e_time - s_time))
      return nde
    expanded_nodes = expand(nde)
    number_of_expansion += 1
    tple = tuple(map(tuple, nde.state))
    is_visited.add(tple)

    for child_node in expanded_nodes:
      child_tuple = tuple(map(tuple, child_node.state))
      if(child_tuple not in is_visited):
        child_node.cost = child_node.cost + misplaced_tile(child_node.state)
        q.put(child_node)
    if(q.qsize() > max_queue_size):
      max_queue_size = q.qsize()
  print("queue is empty!")
  return None

In [16]:
state = [[1, 6, 7], [5, 0, 3], [4, 8, 2]]
node = Node(state)
a_star_with_MT(node)

[[1, 6, 7], [5, 0, 3], [4, 8, 2]] cost: 0
[[1, 6, 7], [0, 5, 3], [4, 8, 2]] cost: 6
[[1, 0, 7], [5, 6, 3], [4, 8, 2]] cost: 7
[[1, 6, 7], [5, 3, 0], [4, 8, 2]] cost: 7
[[1, 6, 7], [5, 8, 3], [4, 0, 2]] cost: 8
[[1, 6, 7], [4, 5, 3], [0, 8, 2]] cost: 11
[[0, 6, 7], [1, 5, 3], [4, 8, 2]] cost: 13
[[1, 7, 0], [5, 6, 3], [4, 8, 2]] cost: 14
[[1, 6, 0], [5, 3, 7], [4, 8, 2]] cost: 14
[[1, 6, 7], [5, 3, 2], [4, 8, 0]] cost: 14
[[0, 1, 7], [5, 6, 3], [4, 8, 2]] cost: 15
[[1, 6, 7], [5, 8, 3], [0, 4, 2]] cost: 16
[[1, 6, 7], [5, 8, 3], [4, 2, 0]] cost: 16
[[1, 6, 7], [4, 5, 3], [8, 0, 2]] cost: 17
[[6, 0, 7], [1, 5, 3], [4, 8, 2]] cost: 20
[[1, 7, 3], [5, 6, 0], [4, 8, 2]] cost: 20
[[1, 0, 6], [5, 3, 7], [4, 8, 2]] cost: 21
[[1, 6, 7], [5, 3, 2], [4, 0, 8]] cost: 22
[[5, 1, 7], [0, 6, 3], [4, 8, 2]] cost: 23
[[1, 6, 7], [4, 5, 3], [8, 2, 0]] cost: 23
[[1, 6, 7], [0, 8, 3], [5, 4, 2]] cost: 24
[[1, 6, 7], [5, 8, 0], [4, 2, 3]] cost: 24
[[1, 6, 7], [4, 0, 3], [8, 5, 2]] cost: 24
[[1, 7, 3], [5, 